In [1]:
import tensorflow as tf
import os
import random
import shutil
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
from os import getcwd
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import RMSprop
from shutil import copyfile
import urllib.request



In [4]:
data_dir='Bmw10_ims'
data_ds=tf.keras.preprocessing.image_dataset_from_directory(data_dir,image_size=(224,224))

Found 512 files belonging to 10 classes.


In [5]:
def preprocess_image_input(input_images):
  input_images = input_images.astype('float32')
  output_ims = tf.keras.applications.resnet50.preprocess_input(input_images)
  return output_ims


In [6]:
def feature_extractor(inputs):

  feature_extractor = tf.keras.applications.resnet.ResNet50(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')(inputs)
  return feature_extractor

In [11]:
def classifier(inputs):
    x = tf.keras.layers.GlobalAveragePooling2D()(inputs)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(1024, activation="relu")(x)
    x = tf.keras.layers.Dense(512, activation="relu")(x)
    x = tf.keras.layers.Dense(10, activation="softmax", name="classification")(x)
    return x

In [12]:
def final_model(inputs):

    #resize = tf.keras.layers.UpSampling2D(size=(7,7))(inputs)

    resnet_feature_extractor = feature_extractor(inputs)
    classification_output = classifier(resnet_feature_extractor)

    return classification_output

In [13]:
def define_compile_model():
  inputs = tf.keras.layers.Input(shape=(224,224,3))
  
  classification_output = final_model(inputs) 
  model = tf.keras.Model(inputs=inputs, outputs = classification_output)
 
  model.compile(optimizer='SGD', 
                loss='sparse_categorical_crossentropy',
                metrics = ['accuracy'])
  
  return model

In [14]:
model = define_compile_model()


In [15]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dense_3 (Dense)              (None, 512)               524800    
_________________________________________________________________
classification (Dense)       (None, 10)                5130

In [17]:
model.fit(data_ds,epochs=30)

Epoch 1/30
16/16 [==============================] - 249s 16s/step - loss: 2.0065 - accuracy: 0.3672
Epoch 2/30
16/16 [==============================] - 265s 17s/step - loss: 1.4495 - accuracy: 0.7559
Epoch 3/30
16/16 [==============================] - 265s 16s/step - loss: 0.9698 - accuracy: 0.9336
Epoch 4/30
16/16 [==============================] - 258s 16s/step - loss: 0.6011 - accuracy: 0.9883
Epoch 5/30
16/16 [==============================] - 272s 17s/step - loss: 0.3479 - accuracy: 1.0000
Epoch 6/30
16/16 [==============================] - 279s 17s/step - loss: 0.2107 - accuracy: 1.0000
Epoch 7/30
16/16 [==============================] - 274s 17s/step - loss: 0.1410 - accuracy: 1.0000
Epoch 8/30
16/16 [==============================] - 281s 17s/step - loss: 0.1021 - accuracy: 1.0000
Epoch 9/30
16/16 [==============================] - 277s 17s/step - loss: 0.0751 - accuracy: 1.0000
Epoch 10/30
16/16 [==============================] - 277s 17s/step - loss: 0.0585 - accuracy: 1.0000

In [ ]:
model.evaluate(data_ds)

In [19]:
model.save('BMW_car_multiclass.h5')

C:\Users\LENOVO\AppData\Roaming\Python\Python39\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


In [20]:
model.save_weights('model_weights')